# Objectif : Comparer les produits structurés, effectuer du clustering pour identifier les outliers

Nous allons faire un embedding des descriptions de produits structurés ainsi que des caractéristiques que nous avons scrappé pour déterminer un score de similarité. Ensuite, nous effectuerons des algorithmes de clustering sur ce score pour déterminer qui sont les outliers, sous-entendu les produits structurés complexes.

Pour cela, il faudra faire apparaitre dans le résumé le détail du fonctionnement du produit structuré afin de pouvoir vraiment comparer.

In [11]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI

In [12]:
df=pd.read_csv(os.path.join(os.getcwd(),'output/bdd_DIC.csv')) 

print(list(df.columns))

['code_ISIN', 'nom_du_produit', 'emetteur_du_produit', 'date_emission', 'date_remboursement', 'mention_complexite', 'montant_minimum_investissement', 'niveau_garantie', 'niveau_barriere_desactivante', 'niveau_risque', 'produit_sous_jacent', 'nature_sous_jacent', 'code_ISIN_sous_jacent', 'frais_ponctuels_entree', 'frais_ponctuels_sortie_echeance', 'frais_ponctuels_sortie_anticipee', 'frais_recurrents', 'frais_accessoires', 'performance_tension', 'performance_maximale', 'espérance_maximale_rendement', 'date_actualisation', 'resume_mecanisme', 'resume_detaille_mecanisme', 'complexite_gpt']


In [14]:

# columns = ['code_ISIN', 'nom_du_produit', 'emetteur_du_produit', 'date_emission', 'date_remboursement', 'mention_complexite', 'montant_minimum_investissement', 'niveau_garantie', 'niveau_barriere_desactivante', 'niveau_risque', 'produit_sous_jacent', 'nature_sous_jacent', 'code_ISIN_sous_jacent', 'frais_ponctuels_entree', 'frais_ponctuels_sortie_echeance', 'frais_ponctuels_sortie_anticipee', 'frais_recurrents', 'frais_accessoires', 'performance_tension', 'performance_maximale', 'espérance_maximale_rendement', 'date_actualisation', 'resume_mecanisme', 'resume_detaille_mecanisme', 'complexite_gpt']

pd.options.mode.chained_assignment = None 

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    return s

df["resume_detaille_mecanisme"] = df["resume_detaille_mecanisme"].apply(lambda x: normalize_text(x))

tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["resume_detaille_mecanisme"].apply(lambda x: len(tokenizer.encode(x)))


In [21]:
client = AzureOpenAI(
  api_key = "7e93421f46cd4680831023addcb0f42d",  
  api_version = "2024-02-15-preview",
  azure_endpoint = "https://francecentral-openai.openai.azure.com"
)

def generate_embeddings(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_v2'] = df["resume_mecanisme"].apply(lambda x : generate_embeddings (x, model = "ada-002")) 

In [25]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["understandability"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("understandability", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df, "Le produit est très simple à comprendre, peu risqué, pour tout le monde", top_n=10)

,code_ISIN,nom_du_produit,emetteur_du_produit,date_emission,date_remboursement,mention_complexite,montant_minimum_investissement,niveau_garantie,niveau_barriere_desactivante,niveau_risque,...,performance_tension,performance_maximale,espérance_maximale_rendement,date_actualisation,resume_mecanisme,resume_detaille_mecanisme,complexite_gpt,n_tokens,ada_v2,understandability
5,FR001400DMN1,Autocall Advanced New,BNP Paribas Issuance B.V.,2023-01-16,2033-04-20,Oui,1000 EUR,0,50%,3,...,-92.9%,4.89%,4.89%,2024-02-28,• Rendement basé sur la performance d'un indic...,Le produit Autocall Advanced New est un titre ...,moyenne,167,"[-0.021074334159493446, -0.024622701108455658,...",0.768962
10,XS1816890997,3Y Bullish Note,BNP Paribas Issuance B.V.,2018-11-23,2023-11-02,Oui,NaN,0,0,2,...,-0.35%,21.08%,NaN,2024-02-28,• Rendement basé sur la performance d'un panie...,Le produit 3Y Bullish Note est un certificat é...,moyenne,166,"[-0.025011727586388588, -0.02696940489113331, ...",0.759463
9,XS2469649045,Athena Privilege Action Airbus Février 2028,BNP Paribas Issuance B.V.,2023-04-18,2028-02-21,Oui,Non spécifié,0,60%,1,...,-98.76%,1.31%,Non spécifié,2024-02-28,• Rendement basé sur la performance d'une acti...,Le produit structuré Athena Privilege Action A...,très simple,254,"[-0.02832520566880703, -0.027496904134750366, ...",0.758770
6,XS1288709972,Autocall Advanced New,BNP Paribas Issuance B.V.,2020-01-07,2030-01-16,Oui,NaN,0,70,5,...,-88.88%,26.3%,11.64%,2024-02-28,• Rendement basé sur la performance de l'indic...,Le produit Autocall Advanced New est une note ...,moyenne,257,"[-0.009217488579452038, -0.026440171524882317,...",0.757561
7,FR001400FJO0,Autocall Advanced New,BNP Paribas Issuance B.V.,2023-04-26,2033-04-28,Oui,1000 EUR,0,60%,6,...,-93.29%,15.06%,16.29%,2024-02-28,• Rendement basé sur la performance d'un indic...,Le produit Autocall Advanced New est un titre ...,complexe,309,"[-0.012367368675768375, -0.023692697286605835,...",0.756234
8,XS2354864741,Eagle Plus,BNP Paribas Issuance B.V.,2022-05-05,2032-04-13,Oui,1000 EUR,0,65%,5,...,-86.9%,28.2%,1.39%,2024-02-28,• Rendement basé sur la performance de l'indic...,Le produit Eagle Plus est un certificat émis p...,moyenne,215,"[-0.016694217920303345, -0.02809678576886654, ...",0.755410
4,FR001400IYK1,Autocall,BNP Paribas Issuance B.V.,2023-07-05,2033-08-30,Oui,1000 EUR,0,50%,5,...,-93.35%,9.81%,9.81%,2024-02-28,"• EMTN, titre de créance négociable, sans prot...",Le produit Autocall est un titre de créance né...,moyenne,209,"[-0.0436871200799942, -0.03358481079339981, -0...",0.752762
1,FR001400K8Q0,Autocall,BNP Paribas Issuance B.V.,2023-09-01,2033-11-14,Oui,NaN,0,0.5,7,...,-100%,8.34%,8.34%,2024-02-28,• Rendement basé sur la performance d'un indic...,Le produit Autocall est un titre de créance né...,complexe,256,"[-0.006397421471774578, -0.022237207740545273,...",0.751538
0,FR001400F9X1,Autocall Advanced Privalto,BNP Paribas Issuance B.V.,2023-01-26,2028-04-18,Oui,Non spécifié,0,60%,1,...,-84.79%,1.18%,1.18%,2024-02-28,"• EMTN, titre de créance négociable, sans prot...",Le produit Autocall Advanced Privalto est un t...,moyenne,243,"[-0.038520053029060364, -0.03568173199892044, ...",0.750566
2,FR001400I350,Autocall,BNP Paribas Issuance B.V.,2023-06-12,2028-05-29,Oui,Non spécifié,0,50%,5,...,"-84,54%","13,12%",Non spécifié,2024-02-28,"• EMTN, titre de créance négociable, sans prot...",Le produit Autocall est un titre de créance né...,moyenne,208,"[-0.04852808639407158, -0.03534961864352226, -...",0.749992


# Nous allons maintenant prendre chaque paire de produit et les comparer pour tenter d'identifier les similarités

In [40]:

similarities = {} # (ISIN1, ISIN2) -> score de similarité

isins = list(df.code_ISIN.unique())


for isin1 in isins:
    for isin2 in isins:
        if isin1 != isin2:
            a = df[df.code_ISIN == isin1].iloc[0].ada_v2
            b = df[df.code_ISIN == isin2].iloc[0].ada_v2
            s = cosine_similarity(a, b)
            similarities[(isin1, isin2)] = s
            similarities[(isin2, isin1)] = s
            print(similarities[(isin1, isin2)])

isin = "FR0014000UZ3"

matrix_similarities = np.zeros((len(isins), len(isins)))


0.8492465120906819
0.9732793701326578
0.9359816142556087
0.9780749500373234
0.8683388586457856
0.8847234710872175
0.8861512249138045
0.8873942308555182
0.8803650701808401
0.8294330683599533
0.8492465120906819
0.845396753515489
0.8643498462782717
0.849881946439083
0.885333923135681
0.8949221146471239
0.9751087139764713
0.920327723349733
0.9364720877129268
0.9246944490982105
0.9732793701326578
0.845396753515489
0.9464687174780527
0.9921967048098166
0.8742361430282072
0.8552494037281938
0.8843816936021538
0.8703194134974933
0.8874586083131369
0.836341996984435
0.9359816142556087
0.8643498462782717
0.9464687174780527
0.9438726181252147
0.9111445652845216
0.8605089410991605
0.8902050296181254
0.8732147678822915
0.8822370664787172
0.8515370630521639
0.9780749500373234
0.849881946439083
0.9921967048098166
0.9438726181252147
0.8801498653093871
0.8627883806061118
0.8912174628976502
0.8775518625661584
0.8848782225937559
0.8298460916167332
0.8683388586457856
0.885333923135681
0.8742361430282072
0

0.8953626077832372
0.9034271481372621
0.9117490809286989
0.8729047397193215
0.8847234710872175
0.8949221146471239
0.8552494037281938
0.8605089410991605
0.8627883806061118
0.8886213870686654
0.9055564362578856
0.9536118918752342
0.9069321998455833
0.8660306480927429
0.8861512249138045
0.9751087139764713
0.8843816936021538
0.8902050296181254
0.8912174628976502
0.8953626077832372
0.9055564362578856
0.9376906675553349
0.9583025522271121
0.9125571112976277
0.8873942308555182
0.920327723349733
0.8703194134974933
0.8732147678822915
0.8775518625661584
0.9034271481372621
0.9536118918752342
0.9376906675553349
0.9414632017340688
0.8844694363874193
0.8803650701808401
0.9364720877129268
0.8874586083131369
0.8822370664787172
0.8848782225937559
0.9117490809286989
0.9069321998455833
0.9583025522271121
0.9414632017340688
0.9164427612475263
0.8294330683599533
0.9246944490982105
0.836341996984435
0.8515370630521639
0.8298460916167332
0.8729047397193215
0.8660306480927429
0.9125571112976277
0.884469436387

KeyError: ('FR0014000UZ3', 'FR001400F9X1')